In [26]:
import pandas as pd

In [3]:
df = pd.read_csv('job_data.csv')

In [4]:
df.head()

,company,job_title,location,salary,company_size,company_founded,company_type,company_industry,company_sector,company_revenue,job_description
0,Bluefin Resources4.0 ★,EMPO Reporting Analyst,Sydney,"$120,000",51 to 200 Employees,--,Company - Private,--,--,Unknown / Non-Applicable,Bluefin Resources are seeking an experienced P...
1,Macquarie Group Limited4.0 ★,Junior Data Business Analyst | Commodities and...,Sydney,"$108,167",10000+ Employees,1969,Company - Public,Investment & Asset Management,Finance,$5 to $10 billion (USD),We are a dynamic change team who works collabo...
2,ETM Group Holdings Pty Ltd,Business Analyst,Alexandria,"$90,000",NaN,NaN,NaN,NaN,NaN,NaN,ETM Group Holdings is currently searching for ...
3,Nixil4.0 ★,Junior Data Analyst: Investment Data Managemen...,Sydney,$0,Unknown,--,Company - Private,--,--,Unknown / Non-Applicable,Work options: Hybrid Our client is seeking a d...
4,TPG Telecom4.0 ★,Data & Reporting Analyst,North Sydney,"$78,772",1001 to 5000 Employees,2020,Company - Public,"Cable, Internet & Telephone Providers",Telecommunications,$2 to $5 billion (USD),Exciting time to join a top 100 ASX listed bus...


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1053 entries, 0 to 1322
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_title         1053 non-null   object 
 1   location          1053 non-null   object 
 2   salary            1053 non-null   float64
 3   company_size      1033 non-null   object 
 4   company_founded   1033 non-null   object 
 5   company_type      1033 non-null   object 
 6   company_industry  1033 non-null   object 
 7   company_sector    1033 non-null   object 
 8   company_revenue   1033 non-null   object 
 9   job_description   1053 non-null   object 
 10  salary_hourly     1053 non-null   int64  
 11  company_name      1053 non-null   object 
 12  company_rating    891 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 147.5+ KB


Checking for duplicates

In [66]:
df.duplicated().sum()

19

In [68]:
df = df.drop_duplicates()

In [69]:
df.duplicated().sum()

0

Company name and rating parsing

In [43]:
df['company']

0             Bluefin Resources4.0 ★
1       Macquarie Group Limited4.0 ★
2         ETM Group Holdings Pty Ltd
4                   TPG Telecom4.0 ★
5                  Sydney Water3.5 ★
                    ...             
1316                Curio Group4.3 ★
1317             Bain & Company4.4 ★
1319             edX Boot Camps4.0 ★
1321          Monash University4.1 ★
1322          Monash University4.1 ★
Name: company, Length: 1053, dtype: object

In [53]:
pattern = r'(.+?)(\d+\.\d+ \★)'
df[['company_name', 'company_rating']] = df['company'].str.extract(pattern, expand=True)

# Handle entries without a rating
df['company_name'] = df['company_name'].fillna(df['company'])

#Clean up company names
df['company_name'] = df['company_name'].str.strip()

#Clean up the rating column (remove the star, strip spaces)
df['company_rating'] = df['company_rating'].str.replace('★', '').str.strip()
df['company_rating'] = df['company_rating'].astype(float)

In [ ]:
df.drop('company', axis=1, inplace=True)

Salary parsing

In [9]:
df.dropna(subset=['salary'], inplace = True)

In [24]:
def salary_parse(x):
    x = x.replace('$', '')
    x = x.replace(',', '')
    return x
salary = df['salary'].apply(salary_parse)

In [28]:
df['salary'] = salary

In [32]:
df['salary'] = df['salary'].apply(lambda x: float(x))

In [35]:
df = df[df['salary'] != 0]

In [38]:
df['salary_hourly'] = df['salary'].apply(lambda x: 1 if x < 5000 else 0)

C:\Users\alexs\AppData\Local\Temp\ipykernel_25768\3799611637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_hourly'] = df['salary'].apply(lambda x: 1 if x < 5000 else 0)


Location cleaning

In [74]:
aus_suburb_df = pd.read_excel('georef-australia-state-suburb.xlsx')

In [92]:
nsw_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "1"]['Official Name Suburb'])
vic_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "2"]['Official Name Suburb'])
qld_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "3"]['Official Name Suburb'])
sa_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "4"]['Official Name Suburb'])
wa_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "5"]['Official Name Suburb'])
tas_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "6"]['Official Name Suburb'])
nt_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "7"]['Official Name Suburb'])
act_suburbs = set(aus_suburb_df[aus_suburb_df['Official Code State'] == "8"]['Official Name Suburb'])

In [96]:
state = []
for location in df['location']:
    if location == 'New South Wales' or location in nsw_suburbs:
        state.append('NSW')
    elif location == 'Victoria' or location in vic_suburbs:
        state.append('VIC')
    elif location == 'Queensland' or location in qld_suburbs or location  == 'Brisbane':
        state.append('QLD')
    elif location == 'South Australia' or location in sa_suburbs or location == 'Adelaide':
        state.append('SA')
    elif location == 'Western Australia' or location in wa_suburbs or location == 'Perth':
        state.append('WA')
    elif location == 'Tasmania' or location in tas_suburbs:
        state.append('TAS')
    elif location == 'Northern Territory' or location in nt_suburbs or location == 'Darwin':
        state.append('NT')
    elif location == 'Australian Capital Territory' or location in act_suburbs or location == 'Canberra':
        state.append('ACT')
    elif location == 'Remote':
        state.append('remote')
    else:
        state.append(input(f'The location is {location} what state is it in?' ))

The location is Seven Hills what state is it in?NSW
The location is Fairfield what state is it in?NSW
The location is Fairfield what state is it in?NSW
The location is Australia what state is it in?AUS
The location is Fairfield what state is it in?NSW
The location is St Leonards what state is it in?NSW
The location is St Leonards what state is it in?NSW
The location is Australia what state is it in?AUS
The location is The Rocks what state is it in?NSW
The location is Rosebery what state is it in?NSW
The location is Girraween what state is it in?NSW
The location is Australia what state is it in?AUS
The location is St Leonards what state is it in?NSW
The location is Redbank what state is it in?QLD
The location is Southport what state is it in?QLD
The location is Australia what state is it in?AUS
The location is Gold Coast what state is it in?QLD
The location is Gold Coast what state is it in?QLD
The location is Cairns what state is it in?QLD
The location is Parkside what state is it in?S

In [98]:
df['state'] = state

In [110]:
df.loc[df['state'] == '', 'state'] = 'WA'

In [116]:
df.loc[df['company_name'] == 'Map Talent', 'state'] = 'NSW'

Parsing of job description

In [159]:
# Some key skills we'll look for include SQL, Python, excel, spark, aws, r studio, data cleaning, data warehousing, 
# Tableau, Power BI, math, machine learning, statistics, presentation and reporting.

In [164]:
df['python'] = df['job_description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['SQL'] = df['job_description'].apply(lambda x: 1 if 'SQL' in x else 0)
df['excel'] = df['job_description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df['spark'] = df['job_description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df['aws'] = df['job_description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df['R'] = df['job_description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r language' in x.lower() else 0)
df['data cleaning'] = df['job_description'].apply(lambda x: 1 if 'data cleaning' in x.lower() else 0)
df['warehousing'] = df['job_description'].apply(lambda x: 1 if 'warehousing' in x.lower() else 0)
df['tableau'] = df['job_description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
df['power_bi'] = df['job_description'].apply(lambda x: 1 if 'power bi' in x.lower() else 0)
df['math'] = df['job_description'].apply(lambda x: 1 if 'math' in x.lower() else 0)
df['machine_learning'] = df['job_description'].apply(lambda x: 1 if 'machine learning' in x.lower() else 0)
df['statistics'] = df['job_description'].apply(lambda x: 1 if 'statistic' in x.lower() else 0)
df['presentation'] = df['job_description'].apply(lambda x: 1 if 'presentation' in x.lower() else 0)
df['reporting'] = df['job_description'].apply(lambda x: 1 if 'reporting' in x.lower() else 0)

In [182]:
boolean_columns = df.select_dtypes(include=['int']).columns

value_counts_per_column = {col: df[col].value_counts() for col in boolean_columns}

for col, counts in value_counts_per_column.items():
    print(f"Value counts for {col}:\n{counts}\n")

Value counts for salary_hourly:
salary_hourly
0    1000
1      34
Name: count, dtype: int64

Value counts for python:
python
0    819
1    215
Name: count, dtype: int64

Value counts for SQL:
SQL
0    693
1    341
Name: count, dtype: int64

Value counts for excel:
excel
1    611
0    423
Name: count, dtype: int64

Value counts for spark:
spark
0    999
1     35
Name: count, dtype: int64

Value counts for aws:
aws
0    947
1     87
Name: count, dtype: int64

Value counts for R:
R
0    1023
1      11
Name: count, dtype: int64

Value counts for data cleaning:
data cleaning
0    1024
1      10
Name: count, dtype: int64

Value counts for warehousing:
warehousing
0    989
1     45
Name: count, dtype: int64

Value counts for tableau:
tableau
0    895
1    139
Name: count, dtype: int64

Value counts for power_bi:
power_bi
0    849
1    185
Name: count, dtype: int64

Value counts for math:
math
0    909
1    125
Name: count, dtype: int64

Value counts for machine_learning:
machine_learning
0   

In [185]:
df.drop(['spark', 'aws', 'R', 'data cleaning', 'warehousing'], axis=1, inplace=True)

Age of company

In [141]:
def company_age_calc(x):
    if x == '--':
        return None
    elif isinstance(x, str):
        return 2023 - int(x)
    else:
        return x

company_age = df['company_founded'].apply(company_age_calc)

In [155]:
df['company_age'] = company_age

Cleaning company size column

In [205]:
df['company_size'].head()

0       51-200
1       >10000
2          NaN
4    1001-5000
5      Unknown
Name: company_size, dtype: object

In [197]:
df['company_size'].value_counts()

company_size
10000+ Employees           216
1 to 50 Employees          167
Unknown                    163
1001 to 5000 Employees     147
51 to 200 Employees        112
201 to 500 Employees        78
501 to 1000 Employees       66
5001 to 10000 Employees     65
Name: count, dtype: int64

In [200]:
def company_size_clean(x):
    if x == '10000+ Employees':
        return '>10000'
    elif x == '5001 to 10000 Employees':
        return '5001-10000'
    elif x == '1001 to 5000 Employees':
        return '1001-5000'
    elif x == '501 to 1000 Employees':
        return '501-1000'
    elif x == '201 to 500 Employees':
        return '201-500'
    elif x == '51 to 200 Employees':
        return '51-200'
    elif x == '1 to 50 Employees':
        return '1-50'
    elif x == 'Unkown':
        return None
    else:
        return x

In [204]:
company_size = df['company_size'].apply(company_size_clean)
df['company_size'] = company_size

Cleanign company type column

In [207]:
df['company_type'].value_counts()

company_type
Company - Private                 500
Company - Public                  372
Government                         50
College / University               31
Non-profit Organisation            16
Subsidiary or Business Segment     16
Unknown                             9
Contract                            8
Hospital                            6
Private Practice / Firm             4
Self-employed                       2
Name: count, dtype: int64

In [210]:
df['company_type'] = df['company_type'].apply(lambda x: None if x == 'Unknown' else x)

Cleaning company industry column

In [213]:
df['company_industry'].value_counts()

company_industry
--                                         310
HR Consulting                              119
Investment & Asset Management               55
Banking & Lending                           46
Information Technology Support Services     44
                                          ... 
Pharmaceutical                               1
Mining & Metals                              1
Grantmaking & Charitable Foundations         1
Health Care Products Manufacturing           1
Primary & Secondary Schools                  1
Name: count, Length: 68, dtype: int64

In [219]:
df['company_industry'].isna().sum()

20

In [222]:
df['company_industry'] = df['company_industry'].apply(lambda x: None if x == '--' else x)

Cleaning company sector column

In [223]:
df['company_sector'].value_counts()

company_sector
--                                             310
Human Resources & Staffing                     137
Finance                                        114
Information Technology                         107
Management & Consulting                         51
Manufacturing                                   43
Retail & Wholesale                              35
Education                                       31
Government & Public Administration              29
Media & Communication                           23
Transportation & Logistics                      18
Energy, Mining, Utilities                       17
Insurance                                       17
Real Estate                                     15
Healthcare                                      14
Construction, Repair & Maintenance Services     11
Aerospace & Defence                              8
Hotel & Travel Accommodation                     6
Telecommunications                               6
Non-profit & NGO

In [225]:
df['company_sector'] = df['company_sector'].apply(lambda x: None if x == '--' else x)

Cleaning company revenue column

In [227]:
df['company_revenue'].value_counts()

company_revenue
Unknown / Non-Applicable            589
$10+ billion (USD)                   94
$2 to $5 billion (USD)               80
$5 to $10 billion (USD)              79
$500 million to $1 billion (USD)     58
$100 to $500 million (USD)           49
$25 to $50 million (USD)             29
$5 to $25 million (USD)              20
$1 to $5 million (USD)               10
Less than $1 million (USD)            6
Name: count, dtype: int64

In [235]:
df['company_revenue'] = df['company_revenue'].apply(lambda x: None if x == 'Unknown / Non-Applicable' else x)

Dropping redundant columns

In [236]:
df.columns

Index(['job_title', 'location', 'salary', 'company_size', 'company_founded',
       'company_type', 'company_industry', 'company_sector', 'company_revenue',
       'job_description', 'salary_hourly', 'company_name', 'company_rating',
       'state', 'company_age', 'python', 'SQL', 'excel', 'tableau', 'power_bi',
       'math', 'machine_learning', 'statistics', 'presentation', 'reporting'],
      dtype='object')

In [239]:
df.drop(['location', 'company_founded', 'job_description'], axis=1, inplace=True)

In [241]:
df

,job_title,salary,company_size,company_type,company_industry,company_sector,company_revenue,salary_hourly,company_name,company_rating,...,python,SQL,excel,tableau,power_bi,math,machine_learning,statistics,presentation,reporting
0,EMPO Reporting Analyst,120000.0,51-200,Company - Private,None,None,None,0,Bluefin Resources,4.0,...,0,0,1,0,0,0,0,0,0,1
1,Junior Data Business Analyst | Commodities and...,108167.0,>10000,Company - Public,Investment & Asset Management,Finance,$5 to $10 billion (USD),0,Macquarie Group Limited,4.0,...,0,0,1,0,0,0,0,0,0,0
2,Business Analyst,90000.0,NaN,NaN,NaN,NaN,NaN,0,ETM Group Holdings Pty Ltd,NaN,...,0,0,0,0,0,0,0,0,0,0
4,Data & Reporting Analyst,78772.0,1001-5000,Company - Public,"Cable, Internet & Telephone Providers",Telecommunications,$2 to $5 billion (USD),0,TPG Telecom,4.0,...,0,1,1,0,1,0,0,0,0,1
5,Data Analyst - Remote,118761.0,Unknown,Government,Energy & Utilities,"Energy, Mining, Utilities",None,0,Sydney Water,3.5,...,1,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,Analytics Manager - Retail,142500.0,51-200,Company - Private,None,None,None,0,Bluefin Resources,4.0,...,1,1,0,0,1,0,1,0,0,0
1316,Expression of Interest for Online Facilitators...,111803.0,1-50,Private Practice / Firm,None,None,None,0,Curio Group,4.3,...,0,0,0,0,0,0,0,0,0,0
1317,"Lead, Machine Learning Engineer",80095.0,>10000,Company - Private,Business Consulting,Management & Consulting,$5 to $10 billion (USD),0,Bain & Company,4.4,...,1,1,0,0,0,0,1,1,0,0
1321,PhD Project – Supervised and knowledge-guided ...,30000.0,>10000,College / University,Colleges & Universities,Education,$2 to $5 billion (USD),0,Monash University,4.1,...,0,0,1,0,0,0,1,0,0,0


In [242]:
df.to_csv('job_data_salary_cleaned.csv', index = False)